In [ ]:
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Reshape
from keras.utils import to_categorical
from keras.backend import image_data_format

from sklearn import datasets
from sklearn.cross_validation import train_test_split

from utils.ferloader import load_fer

# Constants
IMAGE_WIDTH = 48
IMAGE_HEIGHT = 48
EMOTIONS = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

# Prepare data
data, target = load_fer("data/fer2013.npz")
data_train, data_test, target_train, target_test = \
  train_test_split(data, target, test_size=0.2, random_state=7, stratify=target)
_, n_features = data.shape

# Prepare targets
onehot_train = to_categorical(target_train)
onehot_test = to_categorical(target_test)

# Build model
model = Sequential()
if image_data_format() == 'channels_first':
    model.add(Reshape((1, IMAGE_HEIGHT, IMAGE_WIDTH), input_shape=(n_features,)))
else:
    model.add(Reshape((IMAGE_HEIGHT, IMAGE_WIDTH, 1), input_shape=(n_features,)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(len(EMOTIONS)))
model.add(Dropout(0.25))
model.add(Activation("softmax"))

# Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Fit model
callbacks_list = [
    ModelCheckpoint('./checkpoints/best_weights.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min'),
    TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True),
    EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3, verbose=1, mode='auto')
]
model.fit(data_train, onehot_train, epochs=10000, batch_size=32, validation_split=0.2, callbacks=callbacks_list)

# Evaluate best model
model.load_weights("best_weights.hdf5")
results = model.evaluate(data_test, onehot_test, batch_size=len(data_test))
print("\n[!] Evaluation results:")
print("{0}: {2:.3f}, {1}: {3:.3f}".format(*model.metrics_names, *results))

Train on 22967 samples, validate on 5742 samples
Epoch 1/10000
22967/22967 [==============================] - 23s 1ms/step - loss: 13.1926 - acc: 0.1541 - val_loss: 13.9911 - val_acc: 0.1318

Epoch 00001: val_loss improved from inf to 13.99106, saving model to ./checkpoints/best_weights.hdf5
Epoch 2/10000
22967/22967 [==============================] - 23s 1ms/step - loss: 13.4892 - acc: 0.1456 - val_loss: 13.9909 - val_acc: 0.1318

Epoch 00002: val_loss improved from 13.99106 to 13.99094, saving model to ./checkpoints/best_weights.hdf5
Epoch 3/10000
22967/22967 [==============================] - 28s 1ms/step - loss: 13.2369 - acc: 0.1412 - val_loss: 13.9911 - val_acc: 0.1318

Epoch 00003: val_loss did not improve
Epoch 4/10000
 4064/22967 [====>.........................] - ETA: 18s - loss: 13.2645 - acc: 0.1427